In [3]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [2]:
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import FAISS

In [4]:
openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_type = os.environ['OPENAI_API_TYPE']
openai.api_version = os.environ['OPENAI_API_VERSION']

text_model = os.environ['TEXT_DAVINCI_MODEL_NAME']
chat_model = os.environ['CHAT_MODEL_NAME']
embedding_model = os.environ['EMBEDDING_MODEL_NAME']

In [5]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()
len(data)

1000

In [6]:
data[0]

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \r\n\r\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \r\n\r\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \r\n\r\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \r\n\r\nQuestions? Please contact us for any inquiries.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0})

In [7]:
def print_embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Total Tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens / 1000 * 0.0004:.6f}')
    
print_embedding_cost(data)

Total Tokens: 176621
Embedding Cost in USD: 0.070648


In [8]:
from langchain.embeddings import OpenAIEmbeddings
# embeddings = OpenAIEmbeddings() # this won't work because only 1 chunk is supported currently.
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1)

In [9]:
vector_store = FAISS.from_documents(data, embeddings)

In [10]:
query = "Do the Cozy Comfort Pullover Set have side pockets?"
docs = vector_store.similarity_search(query)
print(docs[0].page_content)

: 10
name: Cozy Comfort Pullover Set, Stripe
description: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.

Size & Fit
- Pants are Favorite Fit: Sits lower on the waist.
- Relaxed Fit: Our most generous fit sits farthest from the body.

Fabric & Care
- In the softest blend of 63% polyester, 35% rayon and 2% spandex.

Additional Features
- Relaxed fit top with raglan sleeves and rounded hem.
- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.

Imported.


In [14]:

from langchain.chat_models import AzureChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = AzureChatOpenAI(temperature=0,
    max_tokens=500,
    openai_api_base=openai.api_base,
    openai_api_version=openai.api_version,
    deployment_name=chat_model,
    openai_api_key=openai.api_key,
    openai_api_type = openai.api_type    
)

In [17]:
from langchain.chains import RetrievalQA

retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 2})
chain = RetrievalQA.from_chain_type(llm=chat, chain_type="stuff", retriever=retriever)

query = 'Do the Cozy Comfort Pullover Set have side pockets?'
answer = chain.run(query)
print(answer)

The Cozy Comfort Pullover Set, Stripe has side pockets.


In [18]:
query = 'What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?'
answer = chain.run(query)
print(answer)

The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.
